In [3]:
from abc import abstractmethod
from tabicl.prior.dataset import SCMPrior
from tabicl.prior.prior_config import DEFAULT_FIXED_HP, DEFAULT_SAMPLED_HP

import numpy as np
from scipy.stats import loguniform
import joblib
import math

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nested import nested_tensor
from torch.utils.data import IterableDataset

from typing import Optional, Tuple, Dict, Any, Union

from mcpfn.prior.dataset import DummyPrior, MCARPrior
    
mcar_prior = MCARPrior(num_missing=10, max_features = 20, max_seq_len=100)

X, y, d, seq_lens, train_sizes = mcar_prior.get_batch(batch_size = 4)

print(X.shape)
print(seq_lens, train_sizes)

# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.heatmap(torch.isnan(X), cbar=False, cmap='Blues')

torch.Size([4, 2000, 118])
tensor([2000., 2000., 2000., 2000.]) tensor([1990., 1990., 1990., 1990.])


In [2]:
from mcpfn.prior.training_set_generation import TabICLSCMPrior

prior = TabICLSCMPrior(
    batch_size = 4,
    num_samples = 10,
    num_features = 5,
    num_missing = 10,
    device = 'cpu'
)

X_nested, y_nested, d, seq_lens, train_sizes = prior.get_batch()

/Users/jfeit/tabular/.venv/lib/python3.10/site-packages/torch/nested/__init__.py:228: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. We recommend specifying layout=torch.jagged when constructing a nested tensor, as this layout receives active development, has better operator coverage, and works with torch.compile. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/NestedTensorImpl.cpp:182.)
  return _nested.nested_tensor(


In [4]:
178 / 100

1.78

In [15]:
train_sizes

tensor([40, 40, 40, 40])

In [16]:
print(X_nested[0].shape)

torch.Size([50, 13])


### Nested tensors

In [3]:
import torch

a = torch.randn(2, 6)
b = torch.randn(4, 6)

nested_a = torch.nested.nested_tensor([a, b], layout=torch.jagged)

print(nested_a)

NestedTensor(size=(2, j1, 6), offsets=tensor([0, 2, 6]), contiguous=True)


In [6]:
# Create tensors with different lengths in second dimension
c = torch.randn(2, 2, 4)
d = torch.randn(2, 5, 4)
e = torch.randn(2, 3, 4)

# Create nested tensor with jagged second dimension
nested_b = torch.nested.nested_tensor([c, d, e], layout=torch.jagged)

print(nested_b.shape)


torch.Size([3, 2, j3, 4])


In [1]:
from mcpfn.model.layers import OneHotAndLinearBarDistribution
from mcpfn.model.bar_distribution import get_bucket_limits

embed = OneHotAndLinearBarDistribution(borders = get_bucket_limits(num_outputs = 100, ys = y[0]), embed_dim = 10)

  0%|          | 0/1 [00:00<?, ?it/s]

No DDP training
Model has 32161800 parameters.
Automatic Mixed Precision is enabled.
No checkpoint found, starting from scratch.


In [7]:
from mcpfn.model.mcpfn import MCPFN

model_mcpfn = MCPFN(encoder_path = 'src/mcpfn/model/encoder.pth')
torch.compile(model_mcpfn)

ModuleNotFoundError: No module named 'mcpfn.model.embedding'

In [20]:
from mcpfn.model.tabicl import TabICL
import torch

model_tabicl = TabICL()
torch.compile(model_tabicl)
print(f'TabICL parameters: {sum(p.numel() for p in model_tabicl.parameters() if p.requires_grad)}')

from mcpfn.model.mcpfn import MCPFN

model_mcpfn = MCPFN(encoder_path = 'src/mcpfn/model/encoder.pth')
torch.compile(model_mcpfn)
print(f'MCPFN parameters: {sum(p.numel() for p in model_mcpfn.parameters() if p.requires_grad)}')

RuntimeError: Class values must be non-negative.

In [2]:
X, y = torch.randn(1, 2000, 10), torch.randn(1, 2000)
X_test = torch.randn(1, 10, 10)

In [3]:
%load_ext snakeviz

In [12]:
%%snakeviz -t

out = model_tabicl.forward(torch.cat([X, X_test], dim=1), y, d=torch.tensor([10]))

 
*** Profile stats marshalled to file '/var/folders/zc/6nw_r5b13ws3wfcpdxz6bzcw0000gn/T/tmpa43dxoc6'.
Opening SnakeViz in a new tab...
snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fvar%2Ffolders%2Fzc%2F6nw_r5b13ws3wfcpdxz6bzcw0000gn%2FT%2Ftmpa43dxoc6


In [4]:
%%snakeviz -t

out = model_mcpfn.forward(torch.cat([X, X_test], dim=1), y, d=torch.tensor([10]))

 
*** Profile stats marshalled to file '/var/folders/zc/6nw_r5b13ws3wfcpdxz6bzcw0000gn/T/tmpjl55c6fm'.
Opening SnakeViz in a new tab...
snakeviz web server started on 127.0.0.1:8080; enter Ctrl-C to exit
http://127.0.0.1:8080/snakeviz/%2Fvar%2Ffolders%2Fzc%2F6nw_r5b13ws3wfcpdxz6bzcw0000gn%2FT%2Ftmpjl55c6fm
